In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate

class TextCNN(Model):

    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 kernel_sizes=[3, 4, 5],
                 class_num=4, 
                 last_activation='softmax'):  
        super(TextCNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.kernel_sizes = kernel_sizes
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        self.convs = []
        self.max_poolings = []
        for kernel_size in self.kernel_sizes:
            self.convs.append(Conv1D(128, kernel_size, activation='relu'))
            self.max_poolings.append(GlobalMaxPooling1D())
        self.classifier = Dense(self.class_num, activation=self.last_activation) 

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextCNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of TextCNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        # Embedding part can try multichannel as same as origin paper
        embedding = self.embedding(inputs)
        convs = []
        for i in range(len(self.kernel_sizes)):
            c = self.convs[i](embedding)
            c = self.max_poolings[i](c)
            convs.append(c)
        x = Concatenate()(convs)
        output = self.classifier(x)
        return output


In [2]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

max_features = 500
maxlen = 30
batch_size = 64
embedding_dims = 30
epochs = 10

print('Loading data...')
train_path = "../dataset/preprocessed/netflix_train.csv"
val_path = "../dataset/preprocessed/netflix_val.csv"
test_path = "../dataset/preprocessed/netflix_test.csv"
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

x_train = train_df['title'] + " " + train_df['description']
x_val = val_df['title'] + " " + val_df['description']
x_test = test_df['title'] + " " + test_df['description']

y_train = train_df['target_ages']
y_val = val_df['target_ages']
y_test = test_df['target_ages']

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)
x_test = tokenizer.texts_to_sequences(x_test)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

label_encoder = LabelEncoder()
y_train_encoded = to_categorical(label_encoder.fit_transform(y_train), num_classes=4)
y_test_encoded = to_categorical(label_encoder.transform(y_test), num_classes=4)

print(len(x_train), 'train sequences')
print(len(x_val), 'validation sequences')
print(len(x_test), 'test sequences')
print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextCNN(maxlen, max_features, embedding_dims)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
model.fit(x_train, y_train_encoded,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test_encoded))

print('Test...')
result = model.predict(x_test)

Loading data...
6155 train sequences
1319 validation sequences
1320 test sequences
x_train shape: (6155, 30)
x_val shape: (1319, 30)
x_test shape: (1320, 30)
Build model...


Train...
Epoch 1/10


97/97 [==============================] - 2s 8ms/step - loss: 0.5231 - accuracy: 0.4609 - val_loss: 0.4955 - val_accuracy: 0.4652
Epoch 2/10
97/97 [==============================] - 1s 6ms/step - loss: 0.4746 - accuracy: 0.4874 - val_loss: 0.4565 - val_accuracy: 0.5045
Epoch 3/10
97/97 [==============================] - 1s 6ms/step - loss: 0.4361 - accuracy: 0.5503 - val_loss: 0.4505 - val_accuracy: 0.5106
Epoch 4/10
97/97 [==============================] - 1s 6ms/step - loss: 0.4181 - accuracy: 0.5807 - val_loss: 0.4520 - val_accuracy: 0.5068
Epoch 5/10
97/97 [==============================] - 1s 5ms/step - loss: 0.4023 - accuracy: 0.6037 - val_loss: 0.4569 - val_accuracy: 0.5038
Epoch 6/10
97/97 [==============================] - 1s 6ms/step - loss: 0.3862 - accuracy: 0.6276 - val_loss: 0.46